In [263]:
import numpy as np
import math
import pandas as pd
import pygambit
import itertools
import tqdm

In [244]:
csv_fname = 'toll_equil_mock_2p'
# csv_fname = 'toll_equil_mock_4p'

In [245]:
df = pd.read_csv(csv_fname + '.csv')

In [246]:
players = sorted(set([s.replace('_revenue', '').replace('_tollprice', '') for s in df.columns]))
players

['et1', 'et2']

In [247]:
rev_dict = dict()
for r in df.to_dict('records'):
    d_key = tuple(sorted([(p, r[p + '_tollprice']) for p in players]))
    rev_dict[d_key] = {p: r[p + '_revenue'] for p in players}

In [248]:
price_opt = dict()
for p in players:
    price_opt[p] = np.sort(np.unique(df[p + '_tollprice']))

In [249]:
def get_payoff(coalitions):
    payoff_shape = tuple(np.prod([len(price_opt[p]) for p in c]) for c in coalitions)
    payoff = [np.empty(shape=payoff_shape, dtype=pygambit.Decimal) for c in coalitions]
    all_actions = [list(itertools.product(*[price_opt[p] for p in c])) for c in coalitions]
    for aas in itertools.product(*[enumerate(c_action) for c_action in all_actions]):
        actions_idxs = tuple(a[0] for a in aas)
        actions = [a[1] for a in aas]
        action = tuple(sorted([(p, t) for (ps, ts) in zip(coalitions, actions) for (p, t) in zip(ps, ts)]))
        revs = rev_dict[action]
        for (i, c) in enumerate(coalitions):
            coal_rev = sum(revs[p] for p in c)
            payoff[i][actions_idxs] = pygambit.Decimal(coal_rev)
    return payoff, all_actions

In [250]:
def get_coalition_value(coalitions):
    payoffs, all_actions = get_payoff(coalitions)
    game = pygambit.Game.from_arrays(*payoffs)
    solver = pygambit.nash.ExternalSimpdivSolver()
    profile = solver.solve(game)
    c_payoff = []
    for i, c in enumerate(coalitions):
        c_payoff.append(profile[0].payoff(str(i + 1)))
    return c_payoff, all_actions, profile[0]

In [251]:
def powerset(lst):
    for i in range(1, len(lst) + 1):
        for x in itertools.combinations(lst, i):
            yield x
            

def generate_coalitions(big_coalition):
    coalitions = [big_coalition]
    for p in players:
        if p not in big_coalition:
            coalitions.append(tuple([p]))
    return coalitions

In [265]:
coalition_values = dict()
for c_i in tqdm.tqdm(powerset(players)):
    c = generate_coalitions(c_i)
    val = get_coalition_value(c)[0]
    coalition_values[c_i] = float(val[0])

3it [00:00, 22.79it/s]


In [266]:
for c in coalition_values.keys():
    print(c, coalition_values[c], sum(coalition_values[(p,)] for p in c))

('et1',) 507.5 507.5
('et2',) 312.5 312.5
('et1', 'et2') 1083.3333333333335 820.0


In [267]:
shapley = {p: 0. for p in players}

for ps_perm in itertools.permutations(players):
    curr_gain = 0.
    for i in range(len(players)):
        p_i = ps_perm[i]
        val = coalition_values[tuple(sorted(ps_perm[:i+1]))]
        shapley[p_i] += (val - curr_gain) / math.factorial(len(players))
        curr_gain = val

In [268]:
shapley

{'et1': 639.1666666666667, 'et2': 444.16666666666674}

In [269]:
sum(shapley.values())

1083.3333333333335

In [270]:
sum(coalition_values[(p,)] for p in players)

820.0

## Scrap code below

In [238]:
coal_val_indiv = []

for c_i in tqdm.tqdm(powerset(players)):
    d = dict()
    c = generate_coalitions(c_i)
    val = get_coalition_value(c)[0]
    d['T'] = '$\{' + ','.join('e_{t,' + p[-1] + '}' for p in c_i) + '\}$'
    d['u_big'] = f'{float(val[0]):.1f}'
    for (p, v) in zip(c, val):
        if len(p) == 1:
            p = 'e_{t,' + p[0][-1] + '}'
            d[p] = f'{float(v):.1f}'
    d['tot'] = f'{float(sum(val)):.1f}'
    coal_val_indiv.append(d)

15it [00:11,  1.26it/s]


In [239]:
print(pd.DataFrame(coal_val_indiv).set_index('T').style.to_latex())

\begin{tabular}{llllllr}
 & u_big & e_{t,1} & e_{t,2} & e_{t,3} & e_{t,4} & tot \\
T &  &  &  &  &  &  \\
$\{e_{t,1}\}$ & 324.5 & 324.5 & 759.0 & 541.1 & 468.0 & 2092.462970 \\
$\{e_{t,2}\}$ & 759.0 & 324.5 & 759.0 & 541.1 & 468.0 & 2092.462970 \\
$\{e_{t,3}\}$ & 541.1 & 324.5 & 759.0 & 541.1 & 468.0 & 2092.462970 \\
$\{e_{t,4}\}$ & 468.0 & 324.5 & 759.0 & 541.1 & 468.0 & 2092.462970 \\
$\{e_{t,1},e_{t,2}\}$ & 1083.9 & nan & nan & 538.2 & 469.6 & 2091.671388 \\
$\{e_{t,1},e_{t,3}\}$ & 867.8 & nan & 750.2 & nan & 466.3 & 2084.380413 \\
$\{e_{t,1},e_{t,4}\}$ & 792.4 & nan & 759.0 & 541.1 & nan & 2092.462970 \\
$\{e_{t,2},e_{t,3}\}$ & 1300.0 & 324.5 & nan & nan & 468.0 & 2092.462970 \\
$\{e_{t,2},e_{t,4}\}$ & 1226.9 & 324.5 & nan & 541.1 & nan & 2092.462970 \\
$\{e_{t,3},e_{t,4}\}$ & 1023.2 & 326.3 & 754.9 & nan & nan & 2104.385269 \\
$\{e_{t,1},e_{t,2},e_{t,3}\}$ & 1624.5 & nan & nan & nan & 468.0 & 2092.462970 \\
$\{e_{t,1},e_{t,2},e_{t,4}\}$ & 1553.4 & nan & nan & 538.2 & nan & 2091.67

In [242]:
print(pd.DataFrame([{'e': '$e_{t,' + k[-1] + '}$', 's': f'{v:.1f}'} for (k, v) in shapley.items()]).set_index('e').style.to_latex())

\begin{tabular}{ll}
 & s \\
e &  \\
$e_{t,1}$ & 325.6 \\
$e_{t,2}$ & 757.2 \\
$e_{t,3}$ & 547.2 \\
$e_{t,4}$ & 474.4 \\
\end{tabular}



In [258]:
df

,et2_revenue,et2_tollprice,et1_tollprice,et1_revenue
0,78.333333,4,0,0.000000
1,332.500000,14,16,620.000000
2,125.000000,20,16,820.000000
3,0.000000,0,0,0.000000
4,0.000000,20,2,151.428571
...,...,...,...,...
116,92.500000,2,10,312.500000
117,195.000000,4,14,332.500000
118,229.166667,10,10,479.166667
119,105.000000,12,6,382.500000


In [262]:
coalition_values = dict()
for c_i in tqdm.tqdm(powerset(players)):
    c = generate_coalitions(c_i)
    val, all_actions, profile = get_coalition_value(c)
    print(profile)

3it [00:00, 47.97it/s]

<NashProfile for '': [[Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(1, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1)], [Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(1, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1)]]>
<NashProfile for '': [[Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(1, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1)], [Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(1, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1)]]>
<NashProfile for '': [[Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0,